## 🟢 Ejercicio Básico
Refactoriza esta query eliminando la subconsulta correlacionada:
```sql
SELECT p.nombre,
       (SELECT AVG(v.cantidad) FROM fact_ventas v WHERE v.producto_id = p.producto_id) AS promedio
FROM dim_productos p;
```

## 🟠 Ejercicio Intermedio
Optimiza esta consulta con OR:
```sql
SELECT * FROM fact_ventas
WHERE (cliente_id = 10 AND fecha >= '2024-01-01')
   OR (cliente_id = 20 AND fecha >= '2024-06-01');
```
Reescribe con UNION ALL y mide la mejora de performance.

## 🔴 Ejercicio Avanzado
Diagnostica y optimiza esta query que tarda >30 segundos:
```sql
SELECT c.nombre, p.categoria, 
       SUM(v.cantidad * p.precio_unitario) AS ingresos
FROM fact_ventas v
JOIN dim_clientes c ON v.cliente_id = c.cliente_id
JOIN dim_productos p ON v.producto_id = p.producto_id
WHERE MONTH(v.fecha) = 12
  AND UPPER(c.nombre) LIKE '%CORP%'
GROUP BY c.nombre, p.categoria
ORDER BY ingresos DESC;
```

**Análisis requerido:**
1. Plan de ejecución (captura operadores costosos)
2. Identificar violaciones de sargability
3. Proponer 3 índices estratégicos
4. Refactorizar query (eliminar funciones en WHERE)
5. Medir mejora (antes/después en reads y elapsed time)

---

## Técnica 6: Uso Estratégico de CTEs vs Subqueries

**CTE**: SQL Server puede materializarla o no (decisión del optimizador)
**Subquery**: usualmente inlined (integrada en el plan principal)

```sql
-- CTE puede ser más legible pero no siempre más rápido
WITH ventas_grandes AS (
  SELECT cliente_id, SUM(cantidad) AS total
  FROM fact_ventas
  GROUP BY cliente_id
  HAVING SUM(cantidad) > 1000
)
SELECT c.nombre, v.total
FROM dim_clientes c
JOIN ventas_grandes v ON c.cliente_id = v.cliente_id;
```

💡 **Tip**: compara planes de CTE vs subquery. A veces CTE se ejecuta múltiples veces si se referencia más de una vez.

---

## Errores Comunes

❌ **Optimizar sin medir**: "agregué índice" pero no verificaste que se use
❌ **SELECT * en producción**: lees columnas innecesarias, rompe covering indexes
❌ **NOT IN con NULLs**: si subquery puede retornar NULL, NOT IN falla lógicamente (usa NOT EXISTS)
❌ **Ignorar parameter sniffing**: primer valor de parámetro define plan cacheado (puede ser subóptimo para otros valores)
❌ **Demasiados índices**: cada índice cuesta en INSERT/UPDATE, más espacio, stats a mantener

**Siguiente:** `09_observabilidad_monitoreo.ipynb` → métricas y alertas en producción

### Técnica 3: Paginación Eficiente con OFFSET-FETCH

❌ **Malo (paginación naive):**
```sql
SELECT TOP 20 * FROM fact_ventas ORDER BY fecha DESC;
-- Página 1000: lee 20,000 filas y descarta 19,980
```

✅ **Bueno (OFFSET con índice ordenado):**
```sql
SELECT * FROM fact_ventas
ORDER BY fecha DESC
OFFSET 20000 ROWS FETCH NEXT 20 ROWS ONLY;
-- Con índice en (fecha DESC), salta directamente a la fila 20,001
```

### Técnica 4: Evitar Funciones en WHERE (Sargability)

❌ **Malo (función en columna indexada):**
```sql
WHERE YEAR(fecha) = 2024  -- No usa índice en fecha
WHERE UPPER(nombre) = 'ACME'  -- No usa índice en nombre
```

✅ **Bueno (predicado sargable):**
```sql
WHERE fecha >= '2024-01-01' AND fecha < '2025-01-01'  -- Usa índice
WHERE nombre = 'ACME'  -- Usa índice (asume collation case-insensitive)
```

### Técnica 5: Filtrar Antes de JOIN

❌ **Malo (JOIN grande, luego filtrar):**
```sql
SELECT * FROM fact_ventas v
JOIN dim_productos p ON v.producto_id = p.producto_id
WHERE v.fecha >= '2024-01-01';  -- Filtra DESPUÉS de unir 10M filas
```

✅ **Bueno (filtrar primero con CTE/subquery):**
```sql
WITH ventas_2024 AS (
  SELECT * FROM fact_ventas WHERE fecha >= '2024-01-01'  -- Reduce a 100K filas
)
SELECT * FROM ventas_2024 v
JOIN dim_productos p ON v.producto_id = p.producto_id;
```

### Técnica 1: Eliminar Subconsultas Correlacionadas

❌ **Malo (N+1 queries):**
```sql
SELECT c.nombre,
       (SELECT COUNT(*) FROM fact_ventas v WHERE v.cliente_id = c.cliente_id) AS ventas
FROM dim_clientes c;
-- Por cada cliente, ejecuta una subquery → 100 clientes = 100 scans de fact_ventas
```

✅ **Bueno (1 query con JOIN):**
```sql
SELECT c.nombre, COUNT(v.venta_id) AS ventas
FROM dim_clientes c
LEFT JOIN fact_ventas v ON c.cliente_id = v.cliente_id
GROUP BY c.cliente_id, c.nombre;
-- Un solo scan de fact_ventas
```

### Técnica 2: Reescribir OR con UNION

❌ **Malo (OR impide uso eficiente de índices):**
```sql
SELECT * FROM fact_ventas
WHERE cliente_id = 5 OR producto_id = 101;
-- SQL Server no puede usar índices simultáneamente en ambas columnas
```

✅ **Bueno (UNION ALL usa ambos índices):**
```sql
SELECT * FROM fact_ventas WHERE cliente_id = 5
UNION ALL
SELECT * FROM fact_ventas WHERE producto_id = 101 AND cliente_id <> 5;
-- Dos index seeks, luego concatena resultados
```

# 3.7 Query Tuning - Técnicas Avanzadas de Optimización

## 🎯 ¿Para qué?
Ya sabes medir performance, leer planes y crear índices. Ahora: **técnicas quirúrgicas** para consultas problemáticas que no responden a soluciones obvias.

## 📚 ¿Por qué un enfoque sistemático?

Optimizar "por feeling" falla:
- Agregas índice que no se usa
- Refactorizas sin medir (¿mejoró?)
- Optimizas la parte equivocada (el 5% del costo)

**Método científico:**
1. Medir baseline (tiempo, reads, CPU)
2. Identificar cuello de botella (plan de ejecución)
3. Aplicar técnica específica
4. Medir again (¿mejoró? ¿cuánto?)
5. Iterar o revertir

## 🔧 Técnicas de Query Tuning

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.